In [215]:
# importing all the modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from feature_engine.outliers import Winsorizer
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import warnings
import os
warnings.filterwarnings('ignore')

from data_ingestion import fetch_data_sqlalchemy
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

from evidently.report import Report
from evidently.metric_preset import (DataDriftPreset,TargetDriftPreset, DataQualityPreset)


import matplotlib.pyplot as plt

import shap

In [216]:
# connecting to the database
df = fetch_data_sqlalchemy(
    host='localhost',
    port=5432,
    database='postgres',
    user='postgres',
    password='Nihaar6',
    table_name='employee_salaries'
)


In [217]:
# Preprocessing of missing values

# 1. Fill missing 'experience_level' with mode
df['experience_level'] = df['experience_level'].fillna(df['experience_level'].mode()[0])

# 2. Fill missing 'employment_type' with mode
df['employment_type'] = df['employment_type'].fillna(df['employment_type'].mode()[0])

# 3. Fill missing numerical values (if any) with median
num_cols = ['years_experience', 'base_salary', 'bonus', 'stock_options', 'total_salary', 'salary_in_usd', 'conversion_rate', 'adjusted_total_usd']
for col in num_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

In [218]:
df.isna().sum()

job_title             0
experience_level      0
employment_type       0
company_size          0
company_location      0
remote_ratio          0
salary_currency       0
years_experience      0
base_salary           0
bonus                 0
stock_options         0
total_salary          0
salary_in_usd         0
currency              0
conversion_rate       0
adjusted_total_usd    0
dtype: int64

In [219]:
# Removing null values in target
# Step 1: Remove 0 or NaN salaries
df = df[df['adjusted_total_usd'].notna() & (df['adjusted_total_usd'] > 0)]

In [220]:
# Removing Outliers
# Step 2: Clip extreme values (1st and 99th percentiles)
low, high = df['adjusted_total_usd'].quantile([0.01, 0.99])
df['adjusted_total_usd_clipped'] = df['adjusted_total_usd'].clip(lower=low, upper=high)

In [221]:
# Step 3: Log-transform
df['log_salary'] = np.log1p(df['adjusted_total_usd_clipped'])

In [222]:
df.head()

,job_title,experience_level,employment_type,company_size,company_location,remote_ratio,salary_currency,years_experience,base_salary,bonus,stock_options,total_salary,salary_in_usd,currency,conversion_rate,adjusted_total_usd,adjusted_total_usd_clipped,log_salary
0,Data Analyst,Mid,Contract,Medium,Germany,0,INR,13.0,68407.45175,1100.0,19325.0,88832.45175,1065.989421,USD,1.000,88832.451750,88832.451750,11.394519
1,DevOps Engineer,Mid,Contract,Small,India,100,GBP,9.0,64193.11777,2194.0,19164.0,85551.11777,111216.453100,EUR,1.100,94106.229550,94106.229550,11.452190
2,Research Scientist,Lead,Part-time,Medium,Germany,0,EUR,19.0,136071.84290,3206.0,12735.0,152012.84290,167214.127200,EUR,1.100,167214.127200,167214.127200,12.027036
3,Software Engr,Lead,Full-time,Large,India,50,INR,7.0,141850.90530,9594.0,11158.0,162602.90530,19512.348640,USD,1.000,162602.905300,162602.905300,11.999072
4,Software Engr,Lead,Intern,Large,Germany,100,INR,10.0,121841.16320,6796.0,806.0,129443.16320,1553.317959,INR,0.012,1553.317959,1553.317959,7.348792


In [223]:
# Finding Skewness of target feature
df['log_salary'].skew()

-0.7858337272554825

In [224]:
target = 'log_salary'
X=df[['job_title', 'experience_level', 'employment_type',
    'company_size', 'company_location', 'remote_ratio',
    'years_experience', 'salary_currency', 'conversion_rate'
]]
y = df[target]


In [225]:
df.columns

Index(['job_title', 'experience_level', 'employment_type', 'company_size',
       'company_location', 'remote_ratio', 'salary_currency',
       'years_experience', 'base_salary', 'bonus', 'stock_options',
       'total_salary', 'salary_in_usd', 'currency', 'conversion_rate',
       'adjusted_total_usd', 'adjusted_total_usd_clipped', 'log_salary'],
      dtype='object')

In [226]:
# categorizing features into numerical and categorical
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()


Building Pipelines

In [227]:
def preprocessing_pipeline(X, pipeline_path = "preprocess.pkl"):
    numeric_features = X.select_dtypes(include=['number']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()


    winsor = Winsorizer(
    capping_method='quantiles',
    tail='both',
    fold=0.05,
    variables=numeric_features
    )
    df = winsor.fit_transform(X)
    numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('winsor',Winsorizer(
    capping_method='quantiles',
    tail='both',
    fold=0.05,

    ))
    ])
    categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output= False))
    ])
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
    )
    joblib.dump(preprocessor, 'preprocess.pkl')
    print("✅ Preprocessor saved as 'preprocess.pkl'")
    return preprocessor,numeric_features,categorical_features


In [228]:
# Fit and transform
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



In [229]:
# Get feature names after transformation (for analysis or modeling)
def transform_data(df, preprocessor, numeric_features, categorical_features):

    processed = preprocessor.fit_transform(df)

    cat_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)
    all_feature_names = np.concatenate([numeric_features, cat_feature_names])
    return pd.DataFrame(processed,columns=all_feature_names)
# print(all_feature_names)


In [230]:
# 1. Split first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [231]:

# 2. Preprocessing pipeline - fit only on training data
preprocessor, numeric_features, categorical_features = preprocessing_pipeline(X_train)

✅ Preprocessor saved as 'preprocess.pkl'


In [232]:
# 3. Transform splits
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

In [214]:

# If you want feature names:
cat_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)
all_feature_names = np.concatenate([numeric_features, cat_feature_names])
print("All processed feature names:", all_feature_names)

All processed feature names: ['remote_ratio' 'years_experience' 'conversion_rate'
 'job_title_Data Analyst' 'job_title_Data Scienist'
 'job_title_Data Scntist' 'job_title_DevOps Engineer'
 'job_title_Dt Scientist' 'job_title_ML Enginer' 'job_title_ML Engr'
 'job_title_Machine Learning Engr' 'job_title_Research Scientist'
 'job_title_Software Engr' 'job_title_Softwre Engineer'
 'job_title_Sofware Engneer' 'job_title_data analyst'
 'experience_level_Junior' 'experience_level_Lead' 'experience_level_Mid'
 'experience_level_Senior' 'experience_level_junior'
 'experience_level_mid' 'employment_type_Contract'
 'employment_type_Full-time' 'employment_type_Intern'
 'employment_type_Part-time' 'employment_type_full-time'
 'employment_type_part-time' 'company_size_Large' 'company_size_Medium'
 'company_size_Small' 'company_size_large' 'company_size_medium'
 'company_size_small' 'company_location_Canada' 'company_location_Dubai'
 'company_location_Germany' 'company_location_India'
 'company_locat

In [210]:
# transforming X
processed = transform_data(X, preprocessor, numeric_features, categorical_features)

In [233]:
 X_train.columns.tolist()

['job_title',
 'experience_level',
 'employment_type',
 'company_size',
 'company_location',
 'remote_ratio',
 'years_experience',
 'salary_currency',
 'conversion_rate']

In [234]:
# fit transforming X_train and X_test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)



Model Training


In [235]:

models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge': {
        'model': Ridge(),
        'params': {'alpha': [0.1, 1.0, 10.0]}
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'n_estimators': [100, 200], 'max_depth': [None, 10]}
    },
    'XGBoost': {
        'model': XGBRegressor(random_state=42, verbosity=0),
        'params': {'n_estimators': [100, 200], 'max_depth': [3, 6]}
    }
}


In [236]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100

In [237]:

# 1. Train and evaluate all models (using preprocessed data)
def train_and_evaluate_models(models, X_train, y_train, X_test, y_test,preprocessor, save_dir="saved_models",shap_dir="shap_outputs"):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(shap_dir, exist_ok=True)

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("Instilit Salary Prediction")

    results = []
    best_estimators = {}

    for name, mp in models.items():
        print(f"\nTraining {name}...")
        grid = GridSearchCV(mp['model'], mp['params'], cv=3, scoring='r2', n_jobs=-1)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)

        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        metrics = {
        "mae": mae,
        "rmse": rmse,
        "r2": r2,
        "mape": mape
    }

        print(
            f"{name} | MAE: {mae:.2f} | RMSE: {rmse:.2f} | R2: {r2:.3f} | MAPE: {mape:.2f}% | Best Params: {grid.best_params_}")

        results.append({
            "model": name,
            "best_params": grid.best_params_,
            "mae": mae,
            "rmse": rmse,
            "r2": r2,
            "mape": mape
        })

        # Save the best estimator for this model
        model_filename = f"{save_dir}/{name}_best.pkl"
        joblib.dump(grid.best_estimator_, model_filename)
        print(f"Saved {name} model to {model_filename}")

        # Store best estimator in dictionary
        best_estimators[name] = grid.best_estimator_

        # 🧪 MLflow logging
        with mlflow.start_run(run_name=name) as run:
            mlflow.log_params(grid.best_params_)
            mlflow.log_metrics(metrics)

            input_example = X_test.iloc[:3] if hasattr(X_test, "iloc") else X_test[:3]
            signature = infer_signature(X_test, y_pred)

            #mlflow.sklearn.log_model(grid.best_estimator_, "model")

            mlflow.sklearn.log_model(
                grid.best_estimator_,
                name="model",  # use 'name' instead of 'artifact_path'
                input_example=input_example,
                signature=signature
                )

            # ✅ SHAP Explanation
            try:
                explainer = shap.Explainer(grid.best_estimator_, X_val)
                shap_values = explainer(X_val)

                # Plot and save SHAP summary
                shap_path = os.path.join(shap_dir, f"{name}_shap_summary.png")
                plt.figure()
                shap.summary_plot(shap_values, X_val, show=False)
                plt.savefig(shap_path, bbox_inches='tight')
                plt.close()
                mlflow.log_artifact(shap_path, artifact_path="shap_plots")
                print(f"✅ SHAP saved & logged: {shap_path}")

            except Exception as e:
                print(f"⚠️ SHAP failed for {name}: {e}")

    results_df = pd.DataFrame(results)
    print("\n📊 All Model Validation Metrics:")
    print(results_df[["model", "mae", "rmse", "r2", "mape"]].to_string(index=False))


    return results_df, best_estimators




In [238]:

# 2. Select the overall best model
def select_overall_best_model(results_df, best_estimators,X_train_val, y_train_val,preprocessor, metric="rmse"):
    # For RMSE, lower is better; for R2, higher is better
    ascending = True if metric in ["mae", "rmse", "mape"] else False
    # Step 1: Pick best model
    best_row = results_df.sort_values(by=metric, ascending=ascending).iloc[0]
    best_model_name = best_row["model"]
    best_model = best_estimators[best_model_name]

    # printing best model
    print(f"\nOverall Best Model: {best_model_name} | {metric.upper()}: {best_row[metric]:.4f}")

    # Step 2: Retrain on full data
    best_model.fit(X_train_val, y_train_val)

    return best_model, best_model_name, best_row





In [239]:


# 3. Create, retrain, and save the final pipeline
def retrain_and_save_full_pipeline(preprocessor, best_model, X_full, y_full, save_dir, best_model_name):
    # Step 1: Build pipeline
    full_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('Model', best_model)
    ])
    full_pipeline.fit(X_full, y_full)
    # Step 2: Save pipeline locally
    pipeline_path = os.path.join(save_dir, f"final_pipeline_{best_model_name}_new_pipeline.pkl")
    joblib.dump(full_pipeline, pipeline_path)
    print(f"✅ Full pipeline saved at: {pipeline_path}")


    # 🧪 Step 5: Log & register to MLflow
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("Instilit Salary Prediction")
    client = MlflowClient()

    with mlflow.start_run(run_name=f"Final_{best_model_name}") as run:
        run_id = run.info.run_id

        mlflow.log_artifact(pipeline_path, artifact_path="model")
        print(f"🔁 Registering model to Model Registry: {best_model_name}")
        model_uri = f"runs:/{run_id}/model"

        try:
            registered_model = mlflow.register_model(
                model_uri=model_uri,
                name=best_model_name
            )
        except mlflow.exceptions.MlflowException:
            registered_model = client.create_model_version(
                name=best_model_name,
                source=model_uri,
                run_id=run_id
            )

        # 🏷️ Step 6: Promote to staging
        try:
            client.transition_model_version_stage(
                name=best_model_name,
                version=registered_model.version,
                stage="Staging",
                archive_existing_versions=True
            )
            print(f"✅ Model '{best_model_name}' version {registered_model.version} moved to 'Staging'.")
        except Exception as e:
            print(f"⚠️ Transition to 'Staging' failed: {e}")

        print(f"🏃 View run {best_model_name} at: http://localhost:5000/#/experiments/{run.info.experiment_id}/runs/{run_id}")

    return full_pipeline, best_model_name, pipeline_path

In [240]:
# 4. Complete workflow function
def full_model_workflow(models, preprocessor, X_train, X_test, y_train, y_test, save_dir="saved_models", metric="rmse"):
    # (A) Preprocess train and test data for model selection
    #X_train_processed = preprocessor.fit_transform(X_train)
    #X_test_processed = preprocessor.transform(X_test)

    # (B) Train and evaluate all models
    results_df, best_estimators = train_and_evaluate_models(
        models, X_train, y_train, X_val, y_val,preprocessor)

    # (C) Retrain and save the full pipeline on all data
    X_full = pd.concat([X_train, X_test], axis=0)
    y_full = pd.concat([y_train, y_test], axis=0)

    # (D) Select the overall best model
    best_model, best_model_name, best_row = select_overall_best_model(
        results_df, best_estimators,X_full, y_full,preprocessor, metric=metric
    )


    full_pipeline,best_model_name, pipeline_path = retrain_and_save_full_pipeline(
        preprocessor, best_model, X_full, y_full, save_dir, best_model_name
    )

    return full_pipeline, pipeline_path, best_model_name, best_row, results_df

In [241]:
# Example usage:
full_pipeline, pipeline_path, best_model_name, best_row, results_df = full_model_workflow(
    models, preprocessor, X_train, X_test, y_train, y_test, save_dir="saved_models", metric="rmse"
)

# Now you can use full_pipeline or load it from disk:




Training LinearRegression...


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Minfy\Instilit_Salary_Prediction\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Minfy\Instilit_Salary_Prediction\.venv\lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\Minfy\Instilit_Salary_Prediction\.venv\lib\site-packages\sklearn\linear_model\_base.py", line 618, in fit
    X, y = validate_data(
  File "C:\Users\Minfy\Instilit_Salary_Prediction\.venv\lib\site-packages\sklearn\utils\validation.py", line 2971, in validate_data
    X, y = 

In [40]:
loaded_pipeline = joblib.load(pipeline_path)
predictions = loaded_pipeline.predict(X_test)
print(predictions)

[ 9.90567034 14.09966666  7.49408765 ... 11.97773595 11.25495622
 12.22935283]


In [2]:
X_temp, X_test, y_temp, y_test = train_test_split(processed, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)
def generate_and_log_drift_reports(X_train, X_val, X_test, output_dir, feature_names=None):
    def ensure_df(data, feature_names):
        if isinstance(data, pd.DataFrame):
            return data
        return pd.DataFrame(data, columns=feature_names)

    X_train = ensure_df(X_train, feature_names)
    X_val = ensure_df(X_val, feature_names)
    X_test = ensure_df(X_test, feature_names)

    os.makedirs(output_dir, exist_ok=True)

    comparisons = [
        ("train_vs_val", X_train, X_val),
        ("train_vs_test", X_train, X_test),
        ("val_vs_test", X_val, X_test)
    ]

    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    mlflow.set_experiment("Drift")

    with mlflow.start_run(run_name="multi_split_drift_log"):
        for name, ref, curr in comparisons:
            print(f"🚀 Running drift check: {name}")
            report = Report(metrics=[DataDriftPreset()])
            report.run(reference_data=ref, current_data=curr)

            html_path = os.path.join(output_dir, f'{name}.html')
            json_dict = report.as_dict()

            # Save and log HTML
            report.save_html(html_path)
            mlflow.log_artifact(html_path, artifact_path="evidently_html_reports")

            # ✅ Extract and log drift metrics
            drift_result = next(
                (m["result"] for m in json_dict["metrics"] if m.get("metric") == "DataDriftTable"),
                None
            )
            if drift_result:
                mlflow.log_metric(f"{name}_drift_ratio", round(drift_result["share_of_drifted_columns"], 4))

                for feature, vals in drift_result["drift_by_columns"].items():
                    score = vals.get("drift_score")
                    if score is not None:
                        clean_name = feature.replace(" ", "_").replace("(", "").replace(")", "")
                        mlflow.log_metric(f"{name}_{clean_name}", round(score, 4))

            print(f"✅ Logged drift metrics for {name} to MLflow.\n")

        print(f"🎯 All drift reports and metrics logged under run: {mlflow.active_run().info.run_id}")

# Call the function
generate_and_log_drift_reports(X_train, X_val, X_test, output_dir='drift_reports', feature_names=preprocessor.get_feature_names_out())



NameError: name 'processed' is not defined